In [ ]:
# for plotting

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
sns.set_context('poster')
sns.set_style('ticks')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'cmr10'
plt.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
import numpy as np
import xarray as xr
import metpy
from metpy.units import units

- We looked at possible island effect at ENA following mainly the approach of Ghate et al. (2021, JAMC).
- The surface fluxes used in VarAnal are from buoy data, and possibly calculated using buoy near-surface measurements and bulk formulas.

---

In [ ]:
h = 30. # surface elevation of ENA site in m
lat = 39.0916 # deg N
lon = -28.0257 # deg E

In [ ]:
obs_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/obs'
forc_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/forcing'

In [ ]:
varanal = xr.open_dataset(f'{obs_dir}/ena180varanaecmwfC1.c1.20170601.000000.cdf')
va = varanal.assign_coords(time=varanal.time_offset)

In [ ]:
armbe = xr.open_dataset(f'{obs_dir}/enaarmbeatmC1.c1.20180101.003000.nc')
armbecld = xr.open_dataset(f'{obs_dir}/enaarmbecldradC1.c1.20180101.003000.nc')

In [ ]:
time_era5 = np.arange(0, 24.1, 1.)
era5_sfc = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180206-20180208-sfc.nc')
era5_sfc_ena = era5_sfc.sel(longitude=lon, latitude=lat, method='nearest')
era5_sfc_ena = era5_sfc_ena.sortby(era5_sfc_ena.time)

In [ ]:
time_fnl = np.arange(0, 21.1, 3.)
fnl_sfc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_surface.nc')
fnl_sfc2 = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_heightAboveGround.nc')
fnl_sfc_ena = fnl_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_sfc2_ena = fnl_sfc2.sel(longitude=lon+360., latitude=lat, method='nearest')

In [ ]:
time_merra2 = np.arange(0.5, 23.7, 1.0)
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180207.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180207.nc4')
merra2_sfc_ena = merra2_sfc.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc2_ena = merra2_sfc2.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
time = np.arange(0.5, 24.5, 1.)
time_va = np.arange(0, 24.1, 3.)
u10m = armbe.u_wind_sfc.loc['2018-02-07T00:30:00':'2018-02-07T23:30:00']
v10m = armbe.v_wind_sfc.loc['2018-02-07T00:30:00':'2018-02-07T23:30:00']
wdir = metpy.calc.wind_direction(u10m, v10m)
u10m = era5_sfc_ena.u10.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
v10m = era5_sfc_ena.v10.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
wdir_era5 = metpy.calc.wind_direction(u10m, v10m)
u10m = fnl_sfc2_ena.u10
v10m = fnl_sfc2_ena.v10
wdir_fnl = metpy.calc.wind_direction(u10m, v10m)
u10m = merra2_sfc2_ena.U10M
v10m = merra2_sfc2_ena.V10M
wdir_merra2 = metpy.calc.wind_direction(u10m, v10m)
u10m = va.u_srf.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
v10m = va.v_srf.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
wdir_va = metpy.calc.wind_direction(u10m, v10m)
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
ax.plot(time, wdir, color='black', label='ARM MET')
ax.plot(time_era5, wdir_era5, color='black', linestyle='none', marker='+', label='ERA5')
ax.plot(time_fnl, wdir_fnl, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
ax.plot(time_merra2, wdir_merra2, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
ax.plot(time_va, wdir_va, color='black', linestyle='none', marker='^', markersize=10, label='VarAnal')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
plt.yticks((0, 90, 180, 270))
plt.ylim(0, 360)
plt.axhspan(90, 310, color='lightgray', alpha=0.2)
plt.ylabel('Surface Wind Direction')
plt.title('2018-02-07')
plt.legend()
plt.show()

---

In [ ]:
era5_shf = era5_sfc_ena.ishf.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
era5_lhf = (metpy.constants.water_heat_vaporization*era5_sfc_ena.ie.metpy.quantify()).loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
fnl_shf = fnl_sfc_ena.shtfl
fnl_lhf = fnl_sfc_ena.lhtfl
merra2_shf = merra2_sfc_ena.HFLUX
merra2_lhf = merra2_sfc_ena.EFLUX
va_shf = va.SH.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
va_lhf = va.LH.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
ax.plot(time, armbe.sensible_heat_flux_qcecor.loc['2018-02-07T00:30:00':'2018-02-07T23:30:00'], color='red', label='ECOR, sensible')
ax.plot(time, armbe.latent_heat_flux_qcecor.loc['2018-02-07T00:30:00':'2018-02-07T23:30:00'], color='blue', label='ECOR, latent')
ax.plot(time_era5, -era5_shf, color='red', linestyle='none', marker='+', label='ERA5, sensible')
ax.plot(time_era5, -era5_lhf, color='blue', linestyle='none', marker='+', label='ERA5, latent')
ax.plot(time_fnl, fnl_shf, color='red', linestyle='none', marker='*', label='FNL, sensible')
ax.plot(time_fnl, fnl_lhf, color='blue', linestyle='none', marker='*', label='FNL, latent')
ax.plot(time_merra2, merra2_shf, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2, sensible')
ax.plot(time_merra2, merra2_lhf, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2, latent')
ax.plot(time_va, va_shf, color='red', linestyle='none', marker='^', markersize=10, label='VarAnal, sensible')
ax.plot(time_va, va_lhf, color='blue', linestyle='none', marker='^', markersize=10, label='VarAnal, latent')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
plt.ylabel(r'Surface Fluxes (${\rm W/m^2}$)')
# plt.ylim(-200, 100)
plt.title('2018-02-07')
plt.legend(fontsize=10)
plt.show()

---

In [ ]:
era5_sfc = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180215-20180217-sfc.nc')
era5_l = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180215-20180217-ml.nc')
era5_sfc_ena = era5_sfc.sel(longitude=lon, latitude=lat, method='nearest')
era5_l_ena = era5_l.sel(longitude=lon, latitude=lat, method='nearest')
era5_sfc_ena = era5_sfc_ena.sortby(era5_sfc_ena.time)

In [ ]:
time_fnl = np.arange(0, 21.1, 3.)
fnl_sfc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_surface.nc')
fnl_sfc2 = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_heightAboveGround.nc')
fnl_l = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_isobaricInhPa.nc')
fnl_sfc_ena = fnl_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_sfc2_ena = fnl_sfc2.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_l_ena = fnl_l.sel(longitude=lon+360., latitude=lat, method='nearest')

In [ ]:
time_merra2 = np.arange(0.5, 23.7, 1.0)
merra2_l = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst6_3d_ana_Nv.20180216.nc4')
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180216.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180216.nc4')
merra2_l_ena = merra2_l.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc_ena = merra2_sfc.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc2_ena = merra2_sfc2.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
u10m = armbe.u_wind_sfc.loc['2018-02-16T00:30:00':'2018-02-16T23:30:00']
v10m = armbe.v_wind_sfc.loc['2018-02-16T00:30:00':'2018-02-16T23:30:00']
time = np.arange(0.5, 24.5, 1.)
wdir = metpy.calc.wind_direction(u10m, v10m)
u10m = era5_sfc_ena.u10.loc['2018-02-16T00:00:00':'2018-02-17T00:00:00']
v10m = era5_sfc_ena.v10.loc['2018-02-16T00:00:00':'2018-02-17T00:00:00']
wdir_era5 = metpy.calc.wind_direction(u10m, v10m)
u10m = fnl_sfc2_ena.u10
v10m = fnl_sfc2_ena.v10
wdir_fnl = metpy.calc.wind_direction(u10m, v10m)
u10m = merra2_sfc2_ena.U10M
v10m = merra2_sfc2_ena.V10M
wdir_merra2 = metpy.calc.wind_direction(u10m, v10m)
u10m = va.u_srf.loc['2018-02-16T00:00:00':'2018-02-17T00:00:00']
v10m = va.v_srf.loc['2018-02-16T00:00:00':'2018-02-17T00:00:00']
wdir_va = metpy.calc.wind_direction(u10m, v10m)
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
ax.plot(time, wdir, color='black', label='ARM MET')
ax.plot(time_era5, wdir_era5, color='black', linestyle='none', marker='+', label='ERA5')
ax.plot(time_fnl, wdir_fnl, color='black', linestyle='none', marker='*', label='FNL')
ax.plot(time_merra2, wdir_merra2, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
ax.plot(time_va, wdir_va, color='black', linestyle='none', marker='^', markersize=10, label='VarAnal')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
plt.yticks((0, 90, 180, 270))
plt.ylim(0, 360)
plt.axhspan(90, 310, color='lightgray', alpha=0.2)
plt.ylabel('Surface Wind Direction')
plt.title('2018-02-16')
plt.legend()
plt.show()

---

In [ ]:
era5_shf = era5_sfc_ena.ishf.loc['2018-02-16T00:00:00':'2018-02-17T00:00:00']
era5_lhf = (metpy.constants.water_heat_vaporization*era5_sfc_ena.ie.metpy.quantify()).loc['2018-02-16T00:00:00':'2018-02-17T00:00:00']
fnl_shf = fnl_sfc_ena.shtfl
fnl_lhf = fnl_sfc_ena.lhtfl
merra2_shf = merra2_sfc_ena.HFLUX
merra2_lhf = merra2_sfc_ena.EFLUX
va_shf = va.SH.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
va_lhf = va.LH.loc['2018-02-07T00:00:00':'2018-02-08T00:00:00']
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
ax.plot(time, armbe.sensible_heat_flux_qcecor.loc['2018-02-16T00:30:00':'2018-02-16T23:30:00'], color='red', label='ECOR, sensible')
ax.plot(time, armbe.latent_heat_flux_qcecor.loc['2018-02-16T00:30:00':'2018-02-16T23:30:00'], color='blue', label='ECOR, latent')
ax.plot(time_era5, -era5_shf, color='red', linestyle='none', marker='+', label='ERA5, sensible')
ax.plot(time_era5, -era5_lhf, color='blue', linestyle='none', marker='+', label='ERA5, latent')
ax.plot(time_fnl, fnl_shf, color='red', linestyle='none', marker='*', label='FNL, sensible')
ax.plot(time_fnl, fnl_lhf, color='blue', linestyle='none', marker='*', label='FNL, latent')
ax.plot(time_merra2, merra2_shf, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2, sensible')
ax.plot(time_merra2, merra2_lhf, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2, latent')
ax.plot(time_va, va_shf, color='red', linestyle='none', marker='^', markersize=10, label='VarAnal, sensible')
ax.plot(time_va, va_lhf, color='blue', linestyle='none', marker='^', markersize=10, label='VarAnal, latent')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
plt.ylabel(r'Surface Fluxes (${\rm W/m^2}$)')
# plt.ylim(-100, 450)
plt.title('2018-02-16')
plt.legend(fontsize=12)
plt.show()